In [1]:
from llm_dev.ch2.simple_tokenizer import SimpleTokenizerV1
from llm_dev.ch2.read_text import read_text_file
from llm_dev.ch2 import pre_process_text

In [2]:
input_file = "../../../the-verdict.txt"

In [3]:
full_text = read_text_file(input_file)
vocab = pre_process_text.create_vocab(full_text)
tokenizer = SimpleTokenizerV1(vocab)

Number of tokens: 4690
The generated vocabulary size: 1130


In [4]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

Number of tokens: 7


KeyError: 'Hello'

### 2.4 Adding special context tokens

In [5]:
vocab_v2 = pre_process_text.create_vocab_v2(full_text)

Number of tokens: 4690
The generated vocabulary size: 1132


In [6]:
for i, item in enumerate(list(vocab_v2.items())[-5:]):
    print(i, item)

0 ('younger', 1127)
1 ('your', 1128)
2 ('yourself', 1129)
3 ('<|unk|>', 1130)
4 ('<|endoftext|>', 1131)


### 2.5 Byte pair encoding

In [7]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version('tiktoken'))

tiktoken version: 0.9.0


In [8]:
tokenizer = tiktoken.get_encoding("gpt2")

In [9]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPalace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
print(tokenizer.decode(integers))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 11531, 558, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPalace.


#### Excercise 2.1 Byte Pair encoding of unknown words

In [10]:
ids = tokenizer.encode("Akwirw ier")

print(ids)

In [11]:
for single_id in ids:
    print(tokenizer.decode([single_id]))

Ak
w
ir
w
 
ier


In [12]:
tokenizer.decode(ids)

'Akwirw ier'

### Section 2.6: Data Sampling

In [13]:
text = read_text_file(input_file)
enco_text = tokenizer.encode(text)

In [14]:
enc_sample = enco_text[:50]

In [15]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

In [16]:
print(f"x: {x}")
print(f"y:     {y}")

x: [40, 367, 2885, 1464]
y:     [367, 2885, 1464, 1807]


#### Exercise 2.2: Use data loader with the dataset

In [17]:
with open(input_file, "r", encoding="utf-8") as f:
    raw_text = f.read()

In [18]:
from llm_dev.ch2.data_loader import create_dataloader_v1


data_loader = create_dataloader_v1(
    txt=raw_text,
    batch_size=1,
    max_length=4,
    stride=4,
    shuffle=False,
)
data_iter = iter(data_loader)

/Users/mohammad/arangodb/repos/llm-dev/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [19]:
batch = next(data_iter)


In [20]:
print(batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [21]:
batch_2 = next(data_iter)
print(batch_2)

[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


### Section 2.7: Text Embedding
A hands-on example on how embedding layer works.

In [22]:
import torch
vocab_size = 6
output_dim = 3

torch.manual_seed(42)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 1.9269,  1.4873, -0.4974],
        [ 0.4396, -0.7581,  1.0783],
        [ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [-0.2316,  0.0418, -0.2516],
        [ 0.8599, -0.3097, -0.3957]], requires_grad=True)


In [23]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.6866,  0.6105,  1.3347]], grad_fn=<EmbeddingBackward0>)


In [24]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [ 0.8599, -0.3097, -0.3957],
        [ 0.4396, -0.7581,  1.0783]], grad_fn=<EmbeddingBackward0>)


### Section 2.8: Position Encoding

In [25]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [26]:
max_length = 4
dataloader = create_dataloader_v1(
    txt=raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False
)

In [27]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs: \n", inputs)
print("Inputs shape:\n", inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape:
 torch.Size([8, 4])


In [28]:
print(targets)
print(targets.shape)

tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
torch.Size([8, 4])
